In [1]:
trainTestValid_dataset.save_to_disk('./PAN Data/trainTestValid_dataset/')

NameError: name 'trainTestValid_dataset' is not defined

## Download dataset from the folder

In [14]:
from datasets import DatasetDict
# get the dataset from the bucket
trainTestValid_dataset = DatasetDict.load_from_disk('/home/abindal/pan-2024/notebooks/PAN Data/trainTestValid_dataset')

In [15]:
print(trainTestValid_dataset)

DatasetDict({
    train: Dataset({
        features: ['label', 'text', '__index_level_0__'],
        num_rows: 2191014
    })
    test: Dataset({
        features: ['label', 'text', '__index_level_0__'],
        num_rows: 273877
    })
    valid: Dataset({
        features: ['label', 'text', '__index_level_0__'],
        num_rows: 273877
    })
})


In [16]:
# Load the pre-trained model 
#from transformers import RobertaForSequenceClassification
#model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
# You had previously trained a model on the PAN data, so you can load it here
# The model weights have been finetuned using LoRA approach
from transformers import RobertaForSequenceClassification
model = RobertaForSequenceClassification.from_pretrained("train/LoRAFullDataset/")
model.eval()

# Load the tokenizer
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Define the function to preprocess the data
def preprocess_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

# Preprocess the test dataset 
test_dataset = trainTestValid_dataset['test']
test_dataset = test_dataset.map(preprocess_function, batched=True)
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

/home/abindal/pan-2024/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:628: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 273877/273877 [09:52<00:00, 462.48 examples/s]


In [17]:
# Define the function to compute the accuracy
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = (preds == labels).mean()
    return {'accuracy': acc}

In [18]:
# Evaluate the model on the test dataset
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    output_dir='./TestResults',
)

In [21]:
# Create the Trainer to evaluate the model
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    eval_dataset=test_dataset.shuffle(seed=42).select(range(1000)),
)

In [22]:
# Evaluate the model on the test dataset
results = trainer.evaluate()
print(results)

{'eval_loss': 0.2629634737968445, 'eval_accuracy': 0.903, 'eval_runtime': 377.3824, 'eval_samples_per_second': 2.65, 'eval_steps_per_second': 0.331}
